In [2]:
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import metrics
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

# Zadanie 1

In [3]:
(trainX, trainY), (testX, testY) = tf.keras.datasets.mnist.load_data()
trainX = tf.cast(trainX, tf.float32) / 255.
testX = tf.cast(testX, tf.float32) / 255.

In [4]:
sequential_model = keras.Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation="relu"),
        layers.Dense(128, activation="relu"),
        layers.Dense(10, activation="softmax"),
    ]
)
sequential_model.summary()
sequential_model.load_weights("model_w.h5")
sequential_model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[metrics.SparseCategoricalAccuracy()])


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 784)               0         
                                                                 
 dense (Dense)               (None, 128)               100480    
                                                                 
 dense_1 (Dense)             (None, 128)               16512     
                                                                 
 dense_2 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [10]:
preds_sequential = sequential_model.predict(testX)
accuracy_sequential = accuracy_score(testY, preds_sequential.argmax(axis=1))
precision_sequential, recall_sequential, _, _ = precision_recall_fscore_support(testY, preds_sequential.argmax(axis=1), average='macro')
print("Accuracy: ", accuracy_sequential)
print("Precision: ", precision_sequential)
print("Recall: ", recall_sequential)

313/313 [==============================] - 0s 495us/step
Accuracy:  0.9778
Precision:  0.9778581534433096
Recall:  0.9775777296255154


In [6]:
inputs = layers.Input(shape=(28, 28))
flatten = layers.Flatten()(inputs)
dense1 = layers.Dense(128, activation="relu")(flatten)
dense2 = layers.Dense(128, activation="relu")(dense1)
outputs = layers.Dense(10, activation="softmax")(dense2)
functional_model = keras.Model(inputs=inputs, outputs=outputs)
functional_model.summary()
functional_model.load_weights("model_w.h5")
functional_model.compile(
    optimizer="adam", loss="sparse_categorical_crossentropy", metrics=[metrics.SparseCategoricalAccuracy()]
)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28)]          0         
                                                                 
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 dense_3 (Dense)             (None, 128)               100480    
                                                                 
 dense_4 (Dense)             (None, 128)               16512     
                                                                 
 dense_5 (Dense)             (None, 10)                1290      
                                                                 
Total params: 118,282
Trainable params: 118,282
Non-trainable params: 0
_________________________________________________________________


In [7]:
preds_functional = functional_model.predict(testX)
accuracy_functional = accuracy_score(testY, preds_functional.argmax(axis=1))
precision_functional, recall_functional, _, _ = precision_recall_fscore_support(testY, preds_functional.argmax(axis=1), average='macro')
print("Accuracy: ", accuracy_functional)
print("Precision: ", precision_functional)
print("Recall: ", recall_functional)

313/313 [==============================] - 0s 484us/step
Accuracy:  0.9778
Precision:  0.9778581534433096
Recall:  0.9775777296255154


# Zadanie 2

In [9]:
scalar = layers.Input(shape=(1,), dtype=tf.float32)
batch = layers.Input(shape=(28, 28), dtype=tf.float32)
preds_raw = sequential_model(batch)
preds_sum = layers.Add()([preds_raw, scalar])
raw_output = layers.Softmax()(preds_raw)
sum_output = layers.Softmax()(preds_sum)
model = keras.Model(inputs=[scalar, batch], outputs=[raw_output, sum_output])
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy",
              metrics=[metrics.SparseCategoricalAccuracy()])
raw_preds, sum_preds = model.predict(
    [tf.constant([5.0]*5, dtype=tf.float32), testX[:5]])
print("Raw predictions: ", raw_preds)
print("Sum predictions: ", sum_preds)
print("Raw predictions (indices): ", tf.argmax(raw_preds, axis=1))
print("Sum predictions (indices): ", tf.argmax(sum_preds, axis=1))
print("Test Y values: ", testY[:5])


1/1 [==============================] - 0s 29ms/step
Raw predictions:  [[0.08533798 0.085338   0.08533876 0.08534504 0.08533798 0.08533798
  0.08533798 0.23194955 0.08533799 0.08533865]
 [0.08533676 0.0853368  0.23196906 0.08533682 0.08533676 0.08533676
  0.08533676 0.08533676 0.08533677 0.08533676]
 [0.08536281 0.23155767 0.08537    0.08536318 0.08537012 0.08536298
  0.08536725 0.08548397 0.08539801 0.08536404]
 [0.23194407 0.08533835 0.08534685 0.08533834 0.08533834 0.08533835
  0.08533871 0.08533906 0.08533833 0.08533952]
 [0.08534456 0.08534456 0.08534456 0.08534456 0.23184563 0.08534478
  0.08534456 0.08534483 0.08534476 0.08539723]]
Sum predictions:  [[0.085338   0.085338   0.08533877 0.08534504 0.085338   0.085338
  0.085338   0.23194961 0.085338   0.08533864]
 [0.08533677 0.08533681 0.23196906 0.08533681 0.08533677 0.08533677
  0.08533677 0.08533677 0.08533677 0.08533677]
 [0.08536281 0.23155761 0.08536997 0.08536318 0.0853701  0.08536298
  0.08536725 0.08548395 0.08539799 0.085